In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as trans

trainSet = dsets.MNIST(root='./data', train = True, transform=trans.ToTensor(), download=True)

In [2]:
print('Number of images {}'.format(len(trainSet)))
print('Type {}'.format(type(trainSet[0][0])))
print('Size of each image {}'.format(trainSet[0][0].size()))

Number of images 60000
Type <class 'torch.Tensor'>
Size of each image torch.Size([1, 28, 28])


In [3]:
trainSet[0][0].size() # each image is of size 28 X 28 and is greyscale

torch.Size([1, 28, 28])

In [4]:
class MultiLogisticModel(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MultiLogisticModel, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        
    def forward(self,x):        
        out =self.linear(x)
        return out 

In [5]:
print('Length {} Size {}'.format(len(trainSet), trainSet[0][0].size()))

Length 60000 Size torch.Size([1, 28, 28])


In [6]:
in_dim = 28*28 #input dimensions
out_dim = 10 #output dimensions
model = MultiLogisticModel(in_dim, out_dim) #instantiate model
criterion = nn.CrossEntropyLoss()  #instantiate the loss class
optimiser = torch.optim.SGD(model.parameters(), lr=.1) #instantiate the optimiser class

In [7]:
batchSize = 200
epochs = 1
trainloader = torch.utils.data.DataLoader(dataset=trainSet, batch_size=batchSize, shuffle = True)

In [8]:
for epoch in range(epochs): 

    runningLoss = 0.0
    for i, (images,labels) in enumerate(trainloader):
        images = images.view(-1, 28 * 28)       
        optimiser.zero_grad()
        outputs = model(images)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()                      
        runningLoss += loss.item()
      
    print(runningLoss)  

197.14147725701332


In [9]:
predicted = model.forward(images) 
predicted.size()

torch.Size([200, 10])

In [10]:
print('predictions {}'.format(predicted[0]))
print('labels {}'.format(labels[0]))

predictions tensor([-2.3188, -1.1309,  6.5369,  2.9319, -2.2821, -2.5718, -0.1925, -0.8286,
         1.9347, -1.6721], grad_fn=<SelectBackward>)
labels 2


In [11]:
labels[0:3]

tensor([2, 9, 6])

In [12]:
import numpy as np
def successRate(predicted, labels):
    predict = [np.argmax(p.detach().numpy()) for p in predicted]
    actual =  [labels[i].item() for i in range(len(predicted))]
    correct = [i for i, j in zip(predict, actual) if i == j]
    return(len(correct)/(len(predict)))

In [13]:
successRate(predicted, labels)

0.895

In [14]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=trainSet,  batch_size = 10000, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
output = model(images.view(-1, 28 * 28))
successRate(output,labels)

0.8838

In [15]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=trainSet, batch_size = batchSize, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
output = model(images.view(-1, 28 * 28))
#successRate(output,labels)
predicted = model.forward(images.view(-1, 28 * 28))   
print(predicted[0:3])
print(labels[0:3])
successRate(predicted,labels)

tensor([[ 6.4514, -4.9508,  1.2452, -0.7546, -1.8097,  3.0718,  0.9548, -3.7740,
          2.2809, -2.5286],
        [-0.8575, -3.1152, -0.6729, -3.1439,  2.7192, -0.9009, -0.7080,  1.0994,
          1.4507,  4.6895],
        [ 0.1471, -5.9872,  1.1395,  7.3173, -1.7418,  3.8221, -5.1650, -2.3776,
          4.1774, -1.4668]], grad_fn=<SliceBackward>)
tensor([0, 9, 3])


0.905

In [16]:
pred = [np.argmax(p.detach().numpy()) for p in output] 
incorrect = ([i for i, j in zip(pred, labels) if i != j])

In [18]:
images[0].size()

torch.Size([1, 28, 28])